In [9]:
import pandas as pd

# Define the file paths
file_path_ohs = '/home/dragon/Git/Data/ohs-hc-template_v11.xlsx'
file_path_nte = '/home/dragon/Git/Data/nte-report-41724.xlsx'

# Read the sheet names from the OHS HC Template file
ohs_sheet_names = pd.ExcelFile(file_path_ohs, engine='openpyxl').sheet_names

# Create a dictionary to store the dataframes
ohs_dfs = {}

# Read each sheet into a separate dataframe
for sheet_name in ohs_sheet_names:
    ohs_dfs[sheet_name] = pd.read_excel(file_path_ohs, sheet_name=sheet_name, engine='openpyxl')

# Read the NTE REPORT file starting from the second row
nte_df = pd.read_excel(file_path_nte, skiprows=1, engine='openpyxl')

# Display the sheet names
print("OHS HC Template Sheet Names:")
print(ohs_sheet_names)

# Display the first few rows of each dataframe from the OHS HC Template file
print("\nOHS HC Template DataFrames:")
for sheet_name, df in ohs_dfs.items():
    print(f"\nSheet name: {sheet_name}")
    print(df.head())

# Display the first few rows of the NTE REPORT dataframe
print("\nNTE REPORT DataFrame:")
print(nte_df.head())


OHS HC Template Sheet Names:
['Data Dictionary', 'Positions Instructions', 'Positions Data Template', 'REF - Position Standards', 'REF - Pay Plan Standards', 'REF - Organizational Standards', 'Vacancy Instructions', 'Vacancy Data', 'REF- Vacancy Standards', 'Individual Instructions', 'Individual Data Template', 'REF-Individual Standards', 'External Detailee Instructions', 'External Detailee Data Template', 'Internal Detailee Instructions', 'REF-Detailee Standards', 'Internal Detailee Data Template', 'Direct Support CTR Instructions', 'Direct Support CTR Template', 'Off-site CTR Instructions', 'Off-site CTR Template', 'REF - Contractor Standards']

OHS HC Template DataFrames:

Sheet name: Data Dictionary
   Unnamed: 0                                         Unnamed: 1  Unnamed: 2  \
0         NaN                                                NaN         NaN   
1         NaN  This Excel Workbook contains a suite of  Data ...         NaN   
2         NaN                                  

In [11]:
nte_df.head()

,Pay Period,Last Name,First Name,MI,Level of Education,VET_PREF_RIF,TENURE_GROUP,Not To exceed Date,Employment Type,Action Effective Date,...,Promotion Month,MRA 30,MRA 60/20,MRA 62/5,MRA+10,VERA 50/20,VERA 25,Employee Age,Status All Promotion Eligible,Status All WIGI Eligible
0,5,ANDERSON,KARL,A,Bachelor's degree(13),3,1,,,2024-01-14,...,NaN,11/02/2019,11/02/2023,11/02/2025,11/02/2019,11/02/2013,02/27/2014,60,At Target Grade,At step 10
1,5,BACKER-KRISEL,STEPHANIE,K,Associates degree(10),3,1,,,2024-01-14,...,NaN,08/22/2034,08/22/2038,08/22/2040,08/22/2034,08/22/2028,02/07/2025,45,At Target Grade,JAN-28-24
2,5,BALLERING,KATIE,S,Doctorate degree(21),3,2,,,2024-01-14,...,NaN,04/26/2051,07/03/2041,07/03/2043,07/03/2037,04/26/2041,04/26/2046,42,At Target Grade,MAY-04-25
3,5,BILES,AMBER,DIANE,Master's degree(17),3,3,2024-11-06 00:00:00,,2024-01-14,...,NaN,11/06/2053,11/06/2043,05/19/2037,11/06/2033,11/06/2043,11/06/2048,48,At Target Grade,NOV-04-24
4,5,BLOUNT,BRETT,A,Doctorate degree(21),2,1,,,2024-01-14,...,NaN,03/24/2037,03/24/2027,09/07/2025,09/07/2019,03/24/2027,03/24/2032,60,At Target Grade,At step 10
